In [1]:
import boto3
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()

sagemaker_session = Session()

bucket = sagemaker_session.default_bucket()
prefix = "model-card-sample-notebook"

region = sagemaker_session.boto_region_name
s3 = boto3.client("s3", region_name=region)

In [2]:
import io
import os
import numpy as np
from six.moves.urllib.parse import urlparse
from pprint import pprint
import boto3
import sagemaker
from sagemaker.image_uris import retrieve
import sagemaker.amazon.common as smac
from sagemaker.model_card import (
    ModelCard,
    ModelOverview,
    ObjectiveFunction,
    Function,
    TrainingDetails,
    IntendedUses,
    EvaluationJob,
    AdditionalInformation,
    Metric,
    MetricGroup,
    ModelCardStatusEnum,
    ObjectiveFunctionEnum,
    FacetEnum,
    RiskRatingEnum,
    MetricTypeEnum,
    EvaluationMetricTypeEnum,
)

In [3]:
# synthetic data
raw_data = (
    (0.5, 0),
    (0.75, 0),
    (1.0, 0),
    (1.25, 0),
    (1.50, 0),
    (1.75, 0),
    (2.0, 0),
    (2.25, 1),
    (2.5, 0),
    (2.75, 1),
    (3.0, 0),
    (3.25, 1),
    (3.5, 0),
    (4.0, 1),
    (4.25, 1),
    (4.5, 1),
    (4.75, 1),
    (5.0, 1),
    (5.5, 1),
)
training_data = np.array(raw_data).astype("float32")
labels = training_data[:, 1]

# upload data to S3 bucket
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, training_data, labels)
buf.seek(0)
boto3.resource("s3").Bucket(bucket).Object(os.path.join(prefix, "train")).upload_fileobj(buf)

In [20]:
s3_train_data = f"s3://{bucket}/{prefix}/train"
output_location = f"s3://{bucket}/{prefix}/output"
container = retrieve("linear-learner", sagemaker_session.boto_session.region_name)
estimator = sagemaker.estimator.Estimator(
    container,
    role=role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
)
estimator.set_hyperparameters(feature_dim=2, mini_batch_size=10, predictor_type="binary_classifier")
estimator.fit({"train": s3_train_data})
print(f"Training job name: {estimator.latest_training_job.name}")

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: linear-learner-2023-01-20-02-39-16-216


2023-01-20 02:39:16 Starting - Starting the training job...
2023-01-20 02:39:42 Starting - Preparing the instances for training......
2023-01-20 02:40:36 Downloading - Downloading input data...
2023-01-20 02:41:07 Training - Downloading the training image......
2023-01-20 02:42:12 Training - Training image download completed. Training in progress....
2023-01-20 02:42:48 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[01/20/2023 02:42:37 INFO 140290109187904] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'in

In [21]:
model_name = "model-card-test-model"
model = estimator.create_model(name=model_name)
container_def = model.prepare_container_def()
sagemaker_session.create_model(model_name, role, container_def)
print(f"Model name: {model_name}")

INFO:sagemaker:Creating model with name: model-card-test-model


Model name: model-card-test-model


In [24]:
model_overview = ModelOverview.from_model_name(
    model_name=model_name,
    sagemaker_session=sagemaker_session,
    model_description="This is an example binary classification model used for a Python SDK demo of Amazon SageMaker Model Cards.",
    problem_type="Binary Classification",
    algorithm_type="Logistic Regression",
    model_creator="DEMO-ModelCard",
    model_owner="DEMO-ModelCard",
)
print(f"Model id: {model_overview.model_id}")
print(f"Model training images: {model_overview.inference_environment.container_image}")
print(f"Model: {model_overview.model_artifact}")

ValueError: The model has been associated with ['sample-notebook-model-card'] model cards.

In [23]:
objective_function = ObjectiveFunction(
    function=Function(
        function=ObjectiveFunctionEnum.MINIMIZE,
        facet=FacetEnum.LOSS,
    ),
    notes="This is an example objective function.",
)
training_details = TrainingDetails.from_model_overview(
    model_overview=model_overview,
    sagemaker_session=sagemaker_session,
    objective_function=objective_function,
    training_observations="Add model training observations here.",
)
print(f"Training job id: {training_details.training_job_details.training_arn}")
print(
    f"Training image: {training_details.training_job_details.training_environment.container_image}"
)
print("Training Metrics: ")
pprint(
    [
        {"name": i.name, "value": i.value}
        for i in training_details.training_job_details.training_metrics
    ]
)

Training job id: arn:aws:sagemaker:ap-northeast-2:419974056037:training-job/linear-learner-2023-01-20-02-13-11-792
Training image: ['835164637446.dkr.ecr.ap-northeast-2.amazonaws.com/linear-learner:1']
Training Metrics: 
[{'name': 'train:binary_f_beta', 'value': 0.9473684430122375},
 {'name': 'train:progress', 'value': 100.0},
 {'name': 'train:objective_loss', 'value': 0.4976421296596527},
 {'name': 'train:binary_classification_accuracy', 'value': 0.9473684430122375},
 {'name': 'train:recall', 'value': 1.0},
 {'name': 'train:precision', 'value': 0.8999999761581421},
 {'name': 'train:objective_loss:final', 'value': 0.13517042994499207},
 {'name': 'train:throughput', 'value': 344.0403137207031}]


In [14]:
manual_metric_group = MetricGroup(
    name="binary classification metrics",
    metric_data=[Metric(name="accuracy", type=MetricTypeEnum.NUMBER, value=0.5)],
)
example_evaluation_job = EvaluationJob(
    name="Example evaluation job",
    evaluation_observation="Evaluation observations.",
    datasets=["s3://path/to/evaluation/data"],
    metric_groups=[manual_metric_group],
)
evaluation_details = [example_evaluation_job]

In [15]:
intended_uses = IntendedUses(
    purpose_of_model="Test model card.",
    intended_uses="Not used except this test.",
    factors_affecting_model_efficiency="No.",
    risk_rating=RiskRatingEnum.LOW,
    explanations_for_risk_rating="Just an example.",
)
additional_information = AdditionalInformation(
    ethical_considerations="Your model ethical consideration.",
    caveats_and_recommendations="Your model's caveats and recommendations.",
    custom_details={"custom details1": "details value"},
)

In [19]:
model_card_name = "sample-notebook-model-card"
my_card = ModelCard(
    name=model_card_name,
    status=ModelCardStatusEnum.DRAFT,
    model_overview=model_overview,
    training_details=training_details,
    intended_uses=intended_uses,
    evaluation_details=evaluation_details,
    additional_information=additional_information,
    sagemaker_session=sagemaker_session,
)
my_card.create()
print(f"Model card {my_card.name} is successfully created with id {my_card.arn}")

INFO:sagemaker.model_card.model_card:Creating model card with name: sample-notebook-model-card


ConflictException: An error occurred (ConflictException) when calling the CreateModelCard operation: Modelcard arn:aws:sagemaker:ap-northeast-2:419974056037:model-card/sample-notebook-model-card with version 1 already exists

In [17]:
my_card.model_overview.model_description = "the model is updated."
my_card.update()

INFO:sagemaker.model_card.model_card:Updating model card: sample-notebook-model-card
INFO:sagemaker.model_card.model_card:Update model card content


{'content': {'ModelCardArn': 'arn:aws:sagemaker:ap-northeast-2:419974056037:model-card/sample-notebook-model-card',
  'ResponseMetadata': {'RequestId': '2c76d91c-0737-4a7d-92e7-16af9ef5589f',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amzn-requestid': '2c76d91c-0737-4a7d-92e7-16af9ef5589f',
    'content-type': 'application/x-amz-json-1.1',
    'content-length': '102',
    'date': 'Fri, 20 Jan 2023 02:32:53 GMT'},
   'RetryAttempts': 0}}}

In [18]:
model_overview?

Type:           ModelOverview
String form:    <sagemaker.model_card.model_card.ModelOverview object at 0x7f58236a6d90>
File:           ~/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/model_card/model_card.py
Docstring:      An overview of the model.
Init docstring:
Initialize a Model Overview object.

Args:
    model_id (str, optional): A SageMaker Model ARN or non-SageMaker Model ID (default: None).
    model_name (str, optional): A unique name for the model (default: None).
    model_description (str, optional): A description of the model (default: None).
    model_version (int or float, optional): The version of the model (default: None).
    problem_type (str, optional): The type of problem that the model solves. For example, "Binary Classification", "Multiclass Classification", "Linear Regression", "Computer Vision", or "Natural Language Processing" (default: None).
    algorithm_type (str, optional): The algorithm used to solve the problem type (default: None).
   